In [1]:
import pandas as pd
import numpy as np
import re

# Read original rFBA model

PDF file needs to be parsed manually...

(in order to be in accordance with the syntax of 'id' attribute values of the SBML type 'SId' (Reference: L3V2 Section 3.1.7), the metabolites 13PDG, 2PG, and 3PG were renamed to PDG13, PG2, and PG3, respectively.)

In [2]:
#import PyPDF2
#pdfFileObj = open('Covert2002/SI.pdf', 'rb')
#pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#pageObj = pdfReader.getPage(3)
#pageObj.extractText()

In [3]:
# reactions
SI1 = pd.read_csv('SI-1.tsv', sep='\t')

In [4]:
# "superoxide radicals" are not modeled 
SI1.loc[SI1.ID == 'FUMCR', 'Regulatory logic'] = np.nan
# crp regulation is complex and thus split into many separate statements (vide infra)
SI1.drop(SI1[SI1.Gene == 'crp'].index, inplace=True)


In [5]:
# metabolites
SI2 = pd.read_csv('SI-2.tsv', sep='\t')
SI2 = SI2.sort_values(by=['id'])

## Add Amino Acid-producing enzyme and Ribosome

In [6]:
SI1 = SI1.append({'ID': 'ASPT', 'Protein': 'L-aspartase', 'Gene': 'aspA', 'Reaction': 'FUM <-> Z'}, ignore_index=True)
SI1 = SI1.append({'ID': 'R', 'Protein': 'Ribosome', 'Gene': 'rplA, rplB, rplC, rplD, rplE, rplF, rplI, rplJ, rplK, rplL, rplM, rplN, rplO, rplP, rplQ, rplR, rplS, rplT, rplU, rplV, rplW, rplX, rplY, rpmA, rpmB, rpmC, rpmD, rpmE, rpmF, rpmG, rpmH, rpmI, rpmJ, rpsA, rpsB, rpsC, rpsD, rpsE, rpsF, rpsG, rpsH, rpsI, rpsJ, rpsK, rpsL, rpsM, rpsN, rpsO, rpsP, rpsQ, rpsR, rpsS, rpsT, rpsU'}, ignore_index=True)

## Reformulate gene IDs

In [7]:
# Reformulation of "lumped" IDs
SI1.loc[SI1.ID == 'ACEE', 'Gene'] = 'aceE, aceF, lpdA'
SI1.loc[SI1.ID == 'ATPAR', 'Gene'] = 'atpA, atpB, atpC, atpD, atpE, atpF, atpG, atpH, atpI'
SI1.loc[SI1.ID == 'CYDA', 'Gene'] = 'cydA, cydB'
SI1.loc[SI1.ID == 'CYOA', 'Gene'] = 'cyoA, cyoB, cyoC, cyoD'
SI1.loc[SI1.ID == 'FDNG', 'Gene'] = 'fdnG, fdnH, fdnI'
SI1.loc[SI1.ID == 'FDOH', 'Gene'] = 'fdoI, fdoH, fdoG'
SI1.loc[SI1.ID == 'FRDA', 'Gene'] = 'frdA, frdB, frdC, frdD'
SI1.loc[SI1.ID == 'GLPA', 'Gene'] = 'glpA, glpB, glpC'
SI1.loc[SI1.ID == 'NUOA', 'Gene'] = 'nuoA, nuoB, nuoE, nuoF, nuoG, nuoH, nuoI, nuoJ, nuoK, nuoL, nuoM, nuoN'
SI1.loc[SI1.ID == 'PFLA', 'Gene'] = 'pflA, pflB'
SI1.loc[SI1.ID == 'PFLC', 'Gene'] = 'pflC, pflD'
SI1.loc[SI1.ID == 'PNTA1', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'PNTA2', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'SDHA1', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SDHA2', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SUCA', 'Gene'] = 'sucA, sucB, lpdA'
SI1.loc[SI1.ID == 'SUCCR', 'Gene'] = 'sucC, sucD'
SI1.loc[SI1.ID == 'GLCPTS', 'Gene'] = 'ptsG, ptsH, ptsI, crr'
SI1.loc[SI1.ID == 'PIUP2R', 'Gene'] = 'pitA, pitB'
SI1.loc[SI1.ID == 'RIBUPR', 'Gene'] = 'rbsA, rbsB, rbsC, rbsD'

In [8]:
# manual curation
SI1.loc[SI1.Protein == 'Catabolite activator protein', 'Gene'] = 'cra' # 'cra (fruR)' - beide Namen bezeichnen das gleiche Gen
SI1.loc[SI1.ID == 'GLCUP', 'Gene'] = 'galP' # 'galP, etc.' - galP ist ausreichend
SI1.loc[SI1.ID == 'PCKA', 'Gene'] = 'pck' # 'pckA'
SI1.loc[SI1.ID == 'SFCA', 'Gene'] = 'maeA' # 'sfcA'
SI1.loc[SI1.ID == 'GPMBR', 'Gene'] = 'gpmA' # 'sfcA' - gpmB gibt es nicht; gpmA katalysiert die gleiche Reaktion

In [9]:
genes_list = []

for index, value in SI1['Gene'].items():
    if isinstance(value, str):
        for g in value.split(', '):
            if g not in genes_list:
                genes_list.append(g)

# Species

In [10]:
# Split SI2 into intra- and extracellular metabolites

# add column with boolean variable
SI2['external'] = 1
SI2['external'] = SI2['name'].str.contains("External")

# split dataset
extracellular = SI2.query("external")
intracellular = SI2.query("external == False")

## Intracellular Metabolites $\mathcal{X}$

In [11]:
# drop boolean column
intracellular = intracellular.drop(['external'], axis=1)

In [12]:
# add amino acid
intracellular = intracellular.append({'id': 'Z', 'name': 'amino acid'}, ignore_index=True)

In [13]:
# add attributes
intracellular["compartment"] = 'cytosol'
intracellular["constant"] = 'false'
intracellular["boundaryCondition"] = 'false'
intracellular["hasOnlySubstanceUnits"] = 'true'
intracellular["ram:speciesType"] = 'metabolite'

## Extracellular Metabolites $\mathcal{Y}$

In [14]:
# drop boolean column
extracellular = extracellular.drop(['external'], axis=1)

In [15]:
# abundant species
extracellular.at[extracellular['id'] == 'CO2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'HEXT', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'PIxt', 'initialAmount'] = 10000

# potential product are initially zero
extracellular.at[extracellular['id'] == 'ETHxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'FORxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LCTSxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'PYRxt', 'initialAmount'] = 0

# carbon sources and O2: initial amount depends on the scenario; aerobic glucose as default:
extracellular.at[extracellular['id'] == 'ACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'GLCxt', 'initialAmount'] = 100
extracellular.at[extracellular['id'] == 'GLxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'O2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'RIBxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'SUCCxt', 'initialAmount'] = 0

In [16]:
# add attributes
extracellular["compartment"] = 'extracellular'
extracellular["constant"] = 'false'
extracellular["boundaryCondition"] = 'false' # not false for every species
extracellular["hasOnlySubstanceUnits"] = 'true'
extracellular["ram:speciesType"] = 'extracellular'

## Macromolecules $\mathcal{P}$

### Enzymes $\mathcal{E}$

In [17]:
# create copy of first part of SI
enzymes = SI1[['ID', 'Protein', 'Gene', 'Regulatory logic']].copy()

In [18]:
# delete rows without ID (i.e. regulatory proteins), without Gene (i.e. spontaneous) or without Protein (exchange, biomass, maintenance)
enzymes.dropna(subset=['ID', 'Protein', 'Gene'], inplace=True) # in place: Keep df with valid entries in the same variable

In [19]:
# create enzyme IDs ('E_' + reaction id)
enzymes["id"] = 'E_' + enzymes["ID"]
enzymes = enzymes.drop(['ID'], axis=1) # delete old IDs
enzymes = enzymes[['id', 'Protein', 'Gene', 'Regulatory logic']] # order columns

In [20]:
enzymes = enzymes.rename(columns={"Protein": "name"})

In [21]:
# add attributes
enzymes["compartment"] = 'cytosol'
enzymes["constant"] = 'false'
enzymes["boundaryCondition"] = 'false'
enzymes["hasOnlySubstanceUnits"] = 'true'
enzymes["initialAmount"] = 'NaN'
enzymes["ram:molecularWeight"] = None # are set later
enzymes["ram:objectiveWeight"] = None # are set later
enzymes["ram:biomassPercentage"] = 0.0
enzymes["ram:speciesType"] = 'enzyme'
enzymes["degradationRate"] = 0.01

In [22]:
enzymes.at[enzymes["id"] == "R", "degradationRate"] = 0.001

### Regulatory Proteins $\mathcal{RP}$

In [23]:
# create copy of first part of SI
rp = SI1[['ID', 'Protein', 'Gene']].copy()

In [24]:
# keep rows without ID
rp = rp[pd.isnull(rp['ID'])]
rp = rp.drop(['ID'], axis=1)

In [25]:
# define function for create protein names (uppercase) from gene names (lowercase)
def repl_func(m):
    """process regular expression match groups for word upper-casing problem"""
    return m.group(1) + m.group(2).upper()

In [26]:
rp['id_protein'] = rp['Gene'].map(lambda element: re.sub("(^|\s)(\S)", repl_func, element))

In [27]:
# rp id = 'RP_' + protein id
rp["id"] = 'RP_' + rp["id_protein"]
rp = rp[['id', 'Protein', 'Gene']] # order columns

In [28]:
rp.loc[rp.id == 'RP_Fnr', 'id'] = 'RP_FNR'

In [29]:
rp = rp.rename(columns={"Protein": "name"})

In [30]:
# add attributes
rp["compartment"] = 'cytosol'
rp["constant"] = 'false'
rp["boundaryCondition"] = 'false'
rp["hasOnlySubstanceUnits"] = 'true'
rp["initialAmount"] = 'NaN'
rp["ram:biomassPercentage"] = 0.0
rp["ram:molecularWeight"] = None # are set later
rp["ram:objectiveWeight"] = None # are set later
rp["ram:speciesType"] = 'enzyme'
rp["degradationRate"] = 0.2

### Quota $\mathcal{Q}$

In [31]:
quota = SI1[['Reaction']].loc[SI1['Protein'] == "Biomass production flux"]

In [32]:
biomass_rxn = quota['Reaction'].values

In [33]:
stoich = []
spec = []
product = False

for s in biomass_rxn[0].split():
    if '-' in s:
        product = True
    try:
        s = float(s)
        if product:
            stoich.append(s)
        else:
            stoich.append(-s)
    except ValueError:
        if '+' not in s and '-' not in s:
            spec.append(s)

In [34]:
mw_mol = [503.15, 662.42, 741.38, 258.12, 258.12, 228.09, 198.07, 170.06, 183.03, 165.02, 87.05, 805.54, 130.06, 144.08, 763.50, 424.18, 94.98, 663.42, 740.38, 1000]
mw_mmol = [mw/1000 for mw in mw_mol]

In [35]:
biomass = pd.DataFrame({'id': spec, 'stoichiometry': stoich, 'molecular weight [g/mmol]': mw_mmol})

In [36]:
biomass['weight [g]'] = biomass['stoichiometry'] * biomass['molecular weight [g/mmol]']

In [37]:
# calculate weight of biomass
educt_surplus = biomass['weight [g]'].sum() - 1.0 # subtract biomass 'weight'
biomass_weight = -educt_surplus # new biomass weight equals educt surplus

In [38]:
# set weight of 'biomass', which will be reformulated as Quota_rest
biomass.at[biomass['id'] == 'Biomass', 'biomass_weight'] = biomass_weight

Fraction of proteins $\varphi_{\mathrm{protein}}$ is set to 0.55 (for details, see report), so the 'remaining' quota will make up 45 %

In [39]:
# add attributes for quota 'rest'
quota["ID"] = 'Quota_rest'
quota["ram:molecularWeight"] = biomass_weight
quota["ram:objectiveWeight"] = biomass_weight
quota["ram:biomassPercentage"] = 0.45
quota["degradationRate"] = 0.001
quota["initialAmount"] = 0.45

#### adjust protein fraction $\phi^*_{\text{protein}}$
Determine fraction of proteins explicitly modeled using data by *Li et al.* for E. coli on minimal medium

In [40]:
li = pd.read_csv('Li_minimal.tsv', sep='\t')

In [41]:
li = li.rename(columns={"!Protein:Identifier": "protein_id"})
li = li.rename(columns={"!Protein:Name": "protein_name"})

In [42]:
subunits = pd.DataFrame(genes_list, columns=['Gene'])

In [43]:
for unit in subunits['Gene']:
    if unit in li['protein_name'].tolist():
        abundance = li.loc[li['protein_name'] == unit, '!Abundance:[original]'].iloc[0]
        subunits.loc[subunits.Gene == unit, 'Abundance'] = abundance

In [44]:
# manual curation
subunits.loc[subunits.Gene == 'lpdA', 'Abundance'] = li.loc[li['protein_name'] == 'lpd', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'ackA', 'Abundance'] = li.loc[li['protein_name'] == 'AckA', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'atpB', 'Abundance'] = li.loc[li['protein_name'] == 'AtpB', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'eno', 'Abundance'] = li.loc[li['protein_name'] == 'Eno', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'fba', 'Abundance'] = li.loc[li['protein_name'] == 'fbaA', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'fbp', 'Abundance'] = li.loc[li['protein_name'] == 'Fbp', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'icdA', 'Abundance'] = li.loc[li['protein_name'] == 'Icd', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'pflA', 'Abundance'] = li.loc[li['protein_id'] == 'b0902', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'ppsA', 'Abundance'] = li.loc[li['protein_name'] == 'PpsA', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'rbsB', 'Abundance'] = li.loc[li['protein_name'] == 'RbsB', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'glpR', 'Abundance'] = li.loc[li['protein_id'] == 'b3423', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'lacI', 'Abundance'] = li.loc[li['protein_id'] == 'b0345', '!Abundance:[ppm]'].iloc[0]
subunits.loc[subunits.Gene == 'rpiR', 'Abundance'] = li.loc[li['protein_id'] == 'b4089', '!Abundance:[ppm]'].iloc[0]


In [45]:
proteins = SI1[['ID', 'Gene']].copy()

In [46]:
# delete rows without Gene
proteins.dropna(subset=['Gene'], inplace=True) # in place: Keep df with valid entries in the same variable

In [47]:
proteins['Abundance'] = None
proteins['Length'] = None

In [48]:
for index, value in proteins['Gene'].items():
    abundance = 0
    for g in value.split(', '):
        try:
            abundance = abundance + subunits.loc[subunits['Gene'] == g, 'Abundance'].values[0]
        except ValueError:
            print(value)
    proteins['Abundance'][index] = abundance

In [49]:
uniprot = pd.read_csv('uniprot.tsv', sep='\t')

In [50]:
for index, value in proteins['Gene'].items():
    length = 0
    for g in value.split(', '):
        try:
            length = length + uniprot.loc[uniprot['yourlist'] == g, 'Length'].values[0]
        except ValueError:
            pass
    proteins['Length'][index] = length

In [51]:
proteins['model_MW / mmol'] = proteins['Length'] * 0.1331

In [52]:
proteins['Weighted_Abundance'] = proteins['model_MW / mmol'] * proteins['Abundance']

In [53]:
# sum should be 167 mg
adjust_factor = 0.167 / proteins['Weighted_Abundance'].sum()

In [54]:
proteins['Abundance_adjusted'] = proteins['Abundance'] * adjust_factor

In [55]:
proteins['Weighted_Abundance_adjusted'] =  proteins['model_MW / mmol'] * proteins['Abundance_adjusted']

In [56]:
proteins['Weighted_Abundance_adjusted'].sum()

0.16699999999999998

In [ ]:
# Abundance adjusted = initial amount
for index, value in enzymes['Gene'].items():
    enzymes["initialAmount"][index] = proteins.loc[proteins['Gene'] == value, 'Weighted_Abundance_adjusted'].values[0]


In [ ]:
for index, value in rp['Gene'].items():
    rp["initialAmount"][index] = proteins.loc[proteins['Gene'] == value, 'Weighted_Abundance_adjusted'].values[0]


In [63]:
sum_total = li['!Abundance:[original]'].sum()
sum_explicit = subunits['Abundance'].sum()
frac_explicit = sum_explicit / (sum_total - sum_explicit)
frac_adjusted_total = 0.55*(1-frac_explicit)
frac_explicit_total = frac_explicit * 0.55

In [64]:
print("The fraction of protein explicitly modeled in the dataset by Li et al. is %f%%. Therefore, the adjusted protein quota is %f%% and the fraction of explicitly modeled proteins in the model is %f%%" %(frac_explicit*100, frac_adjusted_total*100, (0.55-frac_adjusted_total)*100))

The fraction of protein explicitly modeled in the dataset by Li et al. is 43.954835%. Therefore, the adjusted protein quota is 30.824841% and the fraction of explicitly modeled proteins in the model is 24.175159%


In [68]:
# add protein quota
# weights are determined later: length will be set to median length of all proteins explicitly modeled
quota = quota.append({"ID": 'Quota_protein', "Gene": 'R', "ram:biomassPercentage": frac_adjusted_total, "degradationRate": 0.01, "initialAmount": frac_adjusted_total}, ignore_index=True)

In [69]:
quota['compartment'] = 'cytosol'
quota['ram:molecularWeight'] = 1.0
quota['ram:objectiveWeight'] = 1.0

In [70]:
quota

,Reaction,ID,ram:molecularWeight,ram:objectiveWeight,ram:biomassPercentage,degradationRate,initialAmount,Gene,compartment
0,41.25 ATP + 3.54 NAD + 18.22 NADPH + 0.2 G6P +...,Quota_rest,1.0,1.0,0.450000,0.001,0.450000,NaN,cytosol
1,NaN,Quota_protein,1.0,1.0,0.308248,0.010,0.308248,R,cytosol


# Reactions $\mathcal{R}$

## Metabolic Reactions $\mathcal{R_X}$ and Uptake Reactions $\mathcal{R_Y}$

In [71]:
# create copy of first part of SI
reactions = SI1[['ID', 'Protein', 'Gene', 'Reaction']].copy()

In [72]:
# delete rows without Reaction
reactions.dropna(subset=['Reaction'], inplace=True)

In [73]:
# delete exchange reactions
reactions = reactions[~reactions.Protein.str.contains('exchange')]

In [74]:
# delete maintenance and biomass reactions
reactions.drop(reactions[reactions.ID == 'ATPM'].index, inplace=True)
reactions.drop(reactions[reactions.ID == 'VGRO'].index, inplace=True)

In [75]:
# add reaction for amino acid
reactions = reactions.append({'ID': 'ASPT', 'Protein': 'L-aspartase', 'Reaction': 'FUM <-> Z'}, ignore_index=True)

### turnover rates

In [76]:
reactions_turnover = pd.read_csv('TurnoverRates.tsv', sep='\t')

In [77]:
# add column with boolean variable
reactions_turnover['transport'] = 1
reactions_turnover['transport'] = reactions_turnover['Protein'].str.contains("transport|efflux|uptake|secretion|permease")

# split dataset
turnover_transport = reactions_turnover.query("transport")
turnover_metabolic = reactions_turnover.query("transport == False")

In [78]:
missing = reactions_turnover['kcat_fwd/s'].str.count('kcat').sum()+reactions_turnover['kcat_bwd/s'].str.count('kcat').sum()+reactions_turnover['kcat_fwd/s'].str.count('diffusion').sum()+reactions_turnover['kcat_bwd/s'].str.count('diffusion').sum()

In [79]:
print("Total number of reactions and reverse reactions: %i" %(len(reactions_turnover)+reactions_turnover['reversible'].sum()))
print("Missing values (total): %i (%f %%)" %(missing, missing/(len(reactions_turnover)+reactions_turnover['reversible'].sum())*100))
print("Missing values (metabolic): %i (%f %%)" %(turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()+turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum(), (turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()+turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum())/(len(turnover_metabolic)+turnover_metabolic['reversible'].sum())*100))
print("Missing values (transport): %i (%f %%)" %(missing-turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()-turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum(), (missing-turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()-turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum())/(len(turnover_transport)+turnover_transport['reversible'].sum())*100))

Total number of reactions and reverse reactions: 152
Missing values (total): 37 (24.342105 %)
Missing values (metabolic): 7 (5.737705 %)
Missing values (transport): 30 (100.000000 %)


In [80]:
all_values = []

# Iterate over two given columns only from the dataframe
for column in reactions_turnover[['kcat_fwd/s', 'kcat_bwd/s']]:
    # Select column contents by column name using [] operator
    columnSeriesObj = reactions_turnover[column]
    for v in columnSeriesObj.values:
        try:
            v = float(v)
            if v > 0:
                all_values.append(v)
        except ValueError:
            pass

In [81]:
np.median(all_values)
#np.mean(all_values)

66.6

In [82]:
reactions_turnover = reactions_turnover.replace(to_replace = 'kcat', value = np.median(all_values))
reactions_turnover = reactions_turnover.replace(to_replace = 'diffusion', value = 0.0) # spontaneous rxns

In [83]:
reactions_turnover["kcat_fwd/s"] = pd.to_numeric(reactions_turnover["kcat_fwd/s"])
reactions_turnover["kcat_bwd/s"] = pd.to_numeric(reactions_turnover["kcat_bwd/s"])

In [84]:
reactions_turnover["fbc:geneProductAssociation"] = np.where(pd.isna(reactions_turnover["Gene"]), np.nan, 'E_' + reactions_turnover["Covert ID"])
reactions_turnover['kcat_fwd/min'] = reactions_turnover['kcat_fwd/s'].apply(lambda x: x*60) # conversion from 1/s to 1/min
reactions_turnover['kcat_bwd/min'] = reactions_turnover['kcat_bwd/s'].apply(lambda x: x*60) # conversion from 1/s to 1/min
reactions_turnover["fbc:maintenanceScaling"] = 0.0

## Translation $\mathcal{R_P}$

In [85]:
# Ribosomal activity: 17 amino acids / s
amino_acids_per_min = 1020
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
weight_amino_acid = 0.1331

In [86]:
translation = SI1[['ID', 'Protein', 'Gene', 'Regulatory logic']].copy()

In [87]:
# drop rows without Gene (spontaneous reactions...)
translation.dropna(subset=['Gene'], inplace=True)

In [88]:
# rename old ID column to "Covert ID"
translation = translation.rename(columns={"ID": "Covert ID"})

In [89]:
# create new ID
translation['id_RP'] = translation['Gene'].map(lambda element: re.sub("(^|\s)(\S)", repl_func, element))
translation["ID"] = np.where(pd.isna(translation["Covert ID"]), 'T_' + translation["id_RP"], 'T_' + translation["Covert ID"])


In [90]:
translation.loc[translation.ID == 'T_Fnr', 'ID'] = 'T_FNR'

In [91]:
# create default ID of Boolean control variable
translation["control variable"] = np.where(pd.isna(translation["Covert ID"]), 'qual_con_' + translation["Gene"] + '_bar', 'qual_con_' + translation["Covert ID"] + '_bar')

In [92]:
translation["Length"] = None

In [93]:
for index, value in translation['Gene'].items():
    length = 0
    for g in value.split(', '):
        try:
            length = length + uniprot.loc[uniprot['yourlist'] == g, 'Length'].values[0]
        except ValueError:
            pass
    translation['Length'][index] = length

In [94]:
median_length = translation['Length'].median()

In [95]:
# append translation of protein quota
translation = translation.append({"ID": 'T_Quota_protein', "Length": median_length}, ignore_index=True)

In [96]:
translation["reversible"] = 'false'
translation['ram:kcatForward'] = round(translation['Length'].apply(lambda x: amino_acids_per_min/x),5)
translation['ram:kcatBackward'] = 0.0
translation["fbc:maintenanceScaling"] = 0.0
translation["fbc:geneProductAssociation"] = 'R'

### set weights for macromolecules

In [97]:
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
for index, value in enzymes['id'].items():
    weight = weight_amino_acid * float(translation.loc[translation['ID'] == value.replace('E_', 'T_'), 'Length'].values[0])
    enzymes.at[index, "ram:molecularWeight"] = weight
    enzymes.at[index, "ram:objectiveWeight"] = weight

In [98]:
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
for index, value in rp['id'].items():
    weight = weight_amino_acid * float(translation.loc[translation['ID'] == value.replace('RP_', 'T_'), 'Length'].values[0])
    rp.at[index, "ram:molecularWeight"] = weight
    rp.at[index, "ram:objectiveWeight"] = weight

In [99]:
# now set weights of protein quota
quota.at[quota['ID'] == 'Quota_protein', 'ram:molecularWeight'] = median_length * weight_amino_acid
quota.at[quota['ID'] == 'Quota_protein', 'ram:objectiveWeight'] = median_length * weight_amino_acid

## Maintenance Reaction $\mathcal{R_M}$

In [100]:
# filter maintenance reaction from SI
main_df = SI1.filter(SI1[SI1.ID == 'ATPM'].index, axis = 0)

# Regulated Proteins $p = \mathcal{RE} \cup \mathcal{RP}$

In [101]:
# create copy of first part of SI
regulated = SI1[['ID', 'Protein', 'Gene', 'Regulatory logic']].copy()

In [102]:
# delete rows without Regulation
regulated.dropna(subset=['Regulatory logic'], inplace=True)

In [103]:
# Create IDs of boolean variables representing gene expresssion for regulated proteins
# regulated enzymes: 'G_' + ID (uppercase)
# regulatory proteins: 'G_' + Gene (lowercase)
regulated["bool_ID"] = np.where(pd.isna(regulated["ID"]), 'G_' + regulated["Gene"] + '_bar', 'G_' + regulated["ID"] + '_bar')

## Events and Algebaric Rules

In [104]:
events = pd.DataFrame()

In [105]:
rules = pd.DataFrame()

### Carbon Catabolite Repression and Crp

In [106]:
# Glc_neg defined by Event low_/high_Glc below
# Lactose
events = events.append({'id':'low_LCTS_aux', 'variable':['GLCxt', 'LCTSxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['Lcts_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_LCTS_aux', 'variable':['GLCxt', 'LCTSxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['Lcts_neg_aux'], 'value':['0']}, ignore_index=True)
# Ribose
events = events.append({'id':'low_RIB_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['Rib_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RIB_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['Rib_neg_aux'], 'value':['0']}, ignore_index=True)
# Glycerol
events = events.append({'id':'low_GL_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['Gl_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GL_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['Gl_neg_aux'], 'value':['0']}, ignore_index=True)
# Lactate / Pyruvate
events = events.append({'id':'low_LacPyr_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['LacPyr_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_LacPyr_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['LacPyr_neg_aux'], 'value':['0']}, ignore_index=True)
# Succinate / Ethanol
events = events.append({'id':'low_SuccEth_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['SuccEth_neg_aux'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_SuccEth_aux', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['SuccEth_neg_aux'], 'value':['0']}, ignore_index=True)
# All Carbon
events = events.append({'id':'low_carbon_neg', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt', 'ACxt', 'FORxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['carbon_neg', 'carbon'], 'value':['1', '0']}, ignore_index=True)
events = events.append({'id':'high_carbon_neg', 'variable':['GLCxt', 'LCTSxt', 'RIBxt', 'GLxt', 'LACxt', 'PYRxt', 'SUCCxt', 'ETHxt', 'ACxt', 'FORxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['carbon_neg', 'carbon'], 'value':['0', '1']}, ignore_index=True)


In [107]:
# GLCUP: reset control variable
translation.loc[translation.ID == 'T_GLCUP', 'control variable'] = 'carbon'

# GLCPTS
events = events.append({'id':'low_Mlc', 'variable':['RP_Mlc'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['Mlc_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_Mlc', 'variable':['RP_Mlc'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['Mlc_neg'], 'value':['0']}, ignore_index=True)
# Cra_neg defined below
rules = rules.append({'variable': 'Glcpts_aux1', 'indicators': ['carbon', 'Mlc_neg'], 'relation': '<and/>'}, ignore_index=True)
rules = rules.append({'variable': 'Glcpts_aux2', 'indicators': ['carbon', 'Cra_neg'], 'relation': '<and/>'}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_GLCPTS_bar', 'indicators': ['Glcpts_aux1', 'Glcpts_aux2'], 'relation': '<or/>'}, ignore_index=True)

# Galactose
events = events.append({'id':'low_GalRS', 'variable':['RP_GalR', 'RP_GalS'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['GalRS_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GalRS', 'variable':['RP_GalR', 'RP_GalS'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['GalRS_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'Gal_aux', 'indicators': ['Glc_neg', 'GalRS_neg'], 'relation': '<and/>'}, ignore_index=True)
translation.loc[translation.ID == 'T_GALER', 'control variable'] = 'Gal_aux'
translation.loc[translation.ID == 'T_GALKR', 'control variable'] = 'Gal_aux'
translation.loc[translation.ID == 'T_GALM1R', 'control variable'] = 'Gal_aux'
translation.loc[translation.ID == 'T_GALM2R', 'control variable'] = 'Gal_aux'
translation.loc[translation.ID == 'T_GALTR', 'control variable'] = 'Gal_aux'

# Lactose
events = events.append({'id':'low_LacI', 'variable':['RP_LacI'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['LacI_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_LacI', 'variable':['RP_LacI'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['LacI_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'Lcts_aux', 'indicators': ['Glc_neg', 'LacI_neg'], 'relation': '<and/>'}, ignore_index=True)
translation.loc[translation.ID == 'T_LACYR', 'control variable'] = 'Lcts_aux'
translation.loc[translation.ID == 'T_LACZ', 'control variable'] = 'Lcts_aux'

# Ribose
events = events.append({'id':'low_RbsR', 'variable':['RP_RbsR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['RbsR_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RbsR', 'variable':['RP_RbsR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['RbsR_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'Rib_aux', 'indicators': ['Lcts_neg', 'RbsR_neg'], 'relation': '<and/>'}, ignore_index=True)
translation.loc[translation.ID == 'T_RIBUPR', 'control variable'] = 'Rib_aux'
translation.loc[translation.ID == 'T_RBSK', 'control variable'] = 'Rib_aux'

# Glycerol
# GLPK, GLUPR
events = events.append({'id':'low_GlpR', 'variable':['RP_GlpR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['GlpR_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GlpR', 'variable':['RP_GlpR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['GlpR_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'GG_aux', 'indicators': ['Rib_neg', 'GlpR_neg'], 'relation': '<and/>'}, ignore_index=True)
translation.loc[translation.ID == 'T_GLPK', 'control variable'] = 'GG_aux'
translation.loc[translation.ID == 'T_GLUPR', 'control variable'] = 'GG_aux'
# GLPA (FNR_neg defined below)
rules = rules.append({'variable': 'qual_con_GLPA_bar', 'indicators': ['GG_aux', 'FNR_aux'], 'relation': '<and/>'}, ignore_index=True)
# GLPD
events = events.append({'id':'low_ArcA_GlpR', 'variable':['RP_ArcA', 'RP_GlpR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['Arca_GlpR_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_ArcA_GlpR', 'variable':['RP_ArcA', 'RP_GlpR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['Arca_GlpR_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_GLPD_bar', 'indicators': ['Rib_neg', 'Arca_GlpR_neg'], 'relation': '<and/>'}, ignore_index=True)

# Succinate
# DCTAR (ArcA_neg defined below)
events = events.append({'id':'low_DcuR', 'variable':['RP_DcuR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['DcuR_aux'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_DcuR', 'variable':['RP_DcuR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['DcuR_aux'], 'value':['1']}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_DCTAR_bar', 'indicators': ['LacPyr_neg', 'Arca_neg', 'DcuR_aux'], 'relation': '<and/>'}, ignore_index=True)
# DCUBR
rules = rules.append({'variable': 'qual_con_DCUBR_bar', 'indicators': ['LacPyr_neg', 'FNR_aux', 'DcuR_aux'], 'relation': '<and/>'}, ignore_index=True)

# Acetate (IclR_neg defined below)
rules = rules.append({'variable': 'qual_con_ACS_bar', 'indicators': ['SuccEth_neg', 'IclR_neg'], 'relation': '<and/>'}, ignore_index=True)

# downstream
translation.loc[translation.ID == 'T_ACNAR', 'control variable'] = 'carbon_neg'
translation.loc[translation.ID == 'T_ACNBR', 'control variable'] = 'carbon_neg'
translation.loc[translation.ID == 'T_PGKR', 'control variable'] = 'carbon_neg'


### Cra

In [108]:
events = events.append({'id':'low_flux_FBP', 'variable':['FBP'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['FDP_aux'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_flux_FBP', 'variable':['FBP'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['FDP_aux'], 'value':['1']}, ignore_index=True)

events = events.append({'id':'low_flux_F6P', 'variable':['TKTA2R', 'TKTB2R', 'TALAR', 'TALBR', 'PGIR'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['F6P_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_flux_F6P', 'variable':['TKTA2R', 'TKTB2R', 'TALAR', 'TALBR', 'PGIR'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['F6P_neg'], 'value':['0']}, ignore_index=True)

rules = rules.append({'variable': 'qual_con_cra_bar', 'indicators': ['FDP_aux', 'F6P_neg'], 'relation': '<and/>'}, ignore_index=True)


### PdhR

In [109]:
events = events.append({'id':'low_flux_MAEB_SFCA', 'variable':['MAEB', 'SFCA'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['PYR_aux1'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_flux_MAEB_SFCA', 'variable':['MAEB', 'SFCA'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['PYR_aux1'], 'value':['1']}, ignore_index=True)

events = events.append({'id':'low_flux_DLD1R', 'variable':['DLD1R'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['DLD1R_aux'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_flux_DLD1R', 'variable':['DLD1R'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['DLD1R_aux'], 'value':['1']}, ignore_index=True)

events = events.append({'id':'low_flux_PYR_aux', 'variable':['GLCPTS', 'PYKF', 'PYKA', 'DLD2', 'DCTAR', 'DCUAR', 'DCUBR'], 'relation':'lt', 'threshold':'theta_v', 'assignment':['PYR_neg_aux3'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_flux_PYR_aux', 'variable':['GLCPTS', 'PYKF', 'PYKA', 'DLD2', 'DCTAR', 'DCUAR', 'DCUBR'], 'relation':'geq', 'threshold':'theta_v', 'assignment':['PYR_neg_aux3'], 'value':['0']}, ignore_index=True)

rules = rules.append({'variable': 'qual_con_pdhR_bar', 'indicators': ['PYR_aux1', 'DLD1R_aux', 'PYR_neg_aux3'], 'relation': '<and/>'}, ignore_index=True)


### ADHER

In [110]:
# O2_neg and Cra_neg are defined below
rules = rules.append({'variable': 'O2_Cra_neg', 'indicators': ['O2_neg', 'Cra_neg'], 'relation': '<and/>'}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_ADHER_bar', 'indicators': ['O2_neg', 'O2_Cra_neg'], 'relation': '<or/>'}, ignore_index=True)


### Proteins controlled by two continuous variables

In [111]:
# ACEB
rules = rules.append({'variable': 'qual_con_ACEB_bar', 'indicators': ['ArcA_neg', 'IclR_neg'], 'relation': '<and/>'}, ignore_index=True)

# CYDA
rules = rules.append({'variable': 'qual_con_CYDA_bar', 'indicators': ['ArcA_aux', 'FNR_neg'], 'relation': '<or/>'}, ignore_index=True)

# FRDA
rules = rules.append({'variable': 'qual_con_FRDA_bar', 'indicators': ['FNR_aux', 'DcuR_aux'], 'relation': '<or/>'}, ignore_index=True)

# DCUC, FORUPR, PFLA, PFLC
rules = rules.append({'variable': 'DFPP_aux', 'indicators': ['ArcA_aux', 'FNR_aux'], 'relation': '<or/>'}, ignore_index=True)
translation.loc[translation.ID == 'T_DCUC', 'control variable'] = 'DFPP_aux'
translation.loc[translation.ID == 'T_FORUPR', 'control variable'] = 'DFPP_aux'
translation.loc[translation.ID == 'T_PFLA', 'control variable'] = 'DFPP_aux'
translation.loc[translation.ID == 'T_PFLC', 'control variable'] = 'DFPP_aux'

# CYOA, FUMAR, SDHA1, SDHA2
rules = rules.append({'variable': 'CFSS_aux', 'indicators': ['ArcA_neg', 'FNR_neg'], 'relation': '<and/>'}, ignore_index=True)
translation.loc[translation.ID == 'T_CYOA', 'control variable'] = 'CFSS_aux'
translation.loc[translation.ID == 'T_FUMAR', 'control variable'] = 'CFSS_aux'
translation.loc[translation.ID == 'T_SDHA1', 'control variable'] = 'CFSS_aux'
translation.loc[translation.ID == 'T_SDHA2', 'control variable'] = 'CFSS_aux'

# FadR
events = events.append({'id':'low_AC', 'variable':['ACxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['AC_neg'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_AC', 'variable':['ACxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['AC_neg'], 'value':['0']}, ignore_index=True)
rules = rules.append({'variable': 'qual_con_fadR_bar', 'indicators': ['GLC_aux', 'AC_neg'], 'relation': '<or/>'}, ignore_index=True)


### Proteins controlled by only one continuous variable

In [112]:
# effect of O2 only
events = events.append({'id':'low_O2', 'variable':['O2xt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar', 'O2_neg'], 'value':['1', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_O2', 'variable':['O2xt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar', 'O2_neg'], 'value':['0', '0', '0']}, ignore_index=True)
# effect of SUCC only
events = events.append({'id':'low_SUCC', 'variable':['SUCCxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_SUCC', 'variable':['SUCCxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['1']}, ignore_index=True)
# effect of LCTS only
events = events.append({'id':'low_LCTS', 'variable':['LCTSxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['1', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_LCTS', 'variable':['LCTSxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['0', '0', '0']}, ignore_index=True)
# effect of GL only
events = events.append({'id':'low_GL', 'variable':['GLxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GL', 'variable':['GLxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['0']}, ignore_index=True)
# effect of GLC only
events = events.append({'id':'low_GLC', 'variable':['GLCxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar', 'Glc_neg', 'Glc_aux'], 'value':['1', '1', '0']}, ignore_index=True)
events = events.append({'id':'high_GLC', 'variable':['GLCxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar', 'Glc_neg', 'Glc_aux'], 'value':['0', '0', '1']}, ignore_index=True)
# effect of RIB only
events = events.append({'id':'low_RIB', 'variable':['RIBxt'], 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_RIB', 'variable':['RIBxt'], 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP DcuS only
events = events.append({'id':'low_DcuS', 'variable':['RP_DcuS'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_DcuS', 'variable':['RP_DcuS'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['1']}, ignore_index=True)
# effect of RP FadR only
events = events.append({'id':'low_FadR', 'variable':['RP_FadR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_FadR', 'variable':['RP_FadR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['1']}, ignore_index=True)

# effect of RP IclR only
events = events.append({'id':'low_IclR', 'variable':['RP_IclR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar', 'IclR_neg'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_IclR', 'variable':['RP_IclR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar', 'IclR_neg'], 'value':['0', '0']}, ignore_index=True)
# effect of RP PdhR only
events = events.append({'id':'low_PdhR', 'variable':['RP_PdhR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_PdhR', 'variable':['RP_PdhR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP FNR only
events = events.append({'id':'low_FNR', 'variable':['RP_FNR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar', 'FNR_aux', 'FNR_neg'], 'value':['0', '0', '1', '0', '1']}, ignore_index=True)
events = events.append({'id':'high_FNR', 'variable':['RP_FNR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar', 'FNR_aux', 'FNR_neg'], 'value':['1', '1', '0', '1', '0']}, ignore_index=True)
# effect of RP ArcA only
events = events.append({'id':'low_arcA', 'variable':['RP_ArcA'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar', 'ArcA_aux', 'ArcA_neg'], 'value':['1', '0', '1']}, ignore_index=True)
events = events.append({'id':'high_arcA', 'variable':['RP_ArcA'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar', 'ArcA_aux', 'ArcA_neg'], 'value':['0', '1', '0']}, ignore_index=True)
# effects of Cra only
events = events.append({'id':'low_Cra', 'variable':['RP_Cra'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar', 'Cra_neg'], 'value':['0', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_Cra', 'variable':['RP_Cra'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar', 'Cra_neg'], 'value':['1', '0', '0']}, ignore_index=True)
# effect of RpiR only
events = events.append({'id':'low_RpiR', 'variable':['RP_RpiR'], 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RpiR', 'variable':['RP_RpiR'], 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['0']}, ignore_index=True)

## Qualitative Species

In [113]:
# initialize with gene names of regulatory proteins
qualspec = rp[['Gene']].copy()
qualspec['id'] = qualspec['Gene'] + '_bar'

In [114]:
# gene expression of regulated enzymes
for index, enzyme in enzymes['id'].items():
    if not pd.isna(enzymes['Regulatory logic'][index]):
        ID = enzyme[2:] + '_bar'
        qualspec = qualspec.append({'id': ID}, ignore_index=True)

In [115]:
# LACUP: spontaneous, but regulated
qualspec = qualspec.append({'id': 'LACUP_bar'}, ignore_index=True)

In [116]:
qualspec['max'] = 1

# Parameters

In [117]:
parameters = {'epsilon_trans':0.0001, 'theta_xt':0.00001, 'theta_RP': 0.000001, 'theta_v': 0.00001}

In [118]:
fbc_parameters = []

In [119]:
qual_parameters = []

# Write Model as XML

In [120]:
rdefba = True

In [121]:
if rdefba:
    name = 'ecoli_rdeFBA'
else:
    name = 'ecoli_deFBA'

In [122]:
f = open(name + '.xml', 'w').close()
f = open(name + '.xml', 'w')

## First lines

In [123]:
# XML definition
f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")

39

In [124]:
# SBML definition
if rdefba:
    f.write("<sbml xmlns=\"http://www.sbml.org/sbml/level3/version2/core\" level=\"3\" version=\"2\" xmlns:qual=\"http://www.sbml.org/sbml/level3/version1/qual/version1\" qual:required=\"true\" xmlns:fbc=\"http://www.sbml.org/sbml/level3/version1/fbc/version2\" fbc:required=\"false\">\n")
else:
    f.write("<sbml xmlns=\"http://www.sbml.org/sbml/level3/version2/core\" level=\"3\" version=\"2\" xmlns:fbc=\"http://www.sbml.org/sbml/level3/version1/fbc/version2\" fbc:required=\"false\">\n")


In [125]:
# Model Definition
f.write("<model id=\"" + name + "\" name=\"" + name + "\" fbc:strict=\"false\">\n\n")

66

## List of Compartments

In [126]:
f.write("<listOfCompartments>\n  <compartment id=\"extracellular\" constant=\"true\"/>\n  <compartment id=\"cytosol\" constant=\"true\"/>\n</listOfCompartments>\n\n")

142

## List of Species

In [127]:
f.write("<listOfSpecies>\n")

16

### Metabolites

#### extracellular metabolites $\mathcal{Y}$

In [128]:
for index, value in extracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+extracellular['name'][index]+"\" compartment=\"extracellular\" initialAmount=\""+str(extracellular['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\""+extracellular['boundaryCondition'][index]+"\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"extracellular\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

#### extracellular metabolites $\mathcal{X}$

In [129]:
for index, value in intracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+intracellular['name'][index]+"\" compartment=\"cytosol\" initialAmount=\"0.0\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"metabolite\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

### Macromolecules $\mathcal{P}$

#### Enzymes $\mathcal{E}$ (including Ribosome)

In [130]:
for index, value in enzymes['id'].items():
    f.write("<species id=\""+value+"\" name=\""+enzymes['name'][index]+"\" compartment=\"cytosol\" initialAmount=\""+str(enzymes['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(enzymes['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(enzymes['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"enzyme\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

#### Regulatory Proteins $\mathcal{RP}$

In [131]:
if rdefba:
    for index, value in rp['id'].items():
        f.write("<species id=\""+value+"\" name=\""+rp['name'][index]+"\" compartment=\"cytosol\" initialAmount=\""+str(rp['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(rp['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(rp['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"enzyme\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

#### Quota $\mathcal{Q}$

In [132]:
for index, value in quota['ID'].items():
    f.write("<species id=\""+value+"\" compartment=\"cytosol\" initialAmount=\""+str(quota['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(quota['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(quota['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\""+str(quota['ram:biomassPercentage'][index])+"\" ram:speciesType=\"quota\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")    

In [133]:
f.write("</listOfSpecies>\n\n")

18

## List of Reactions $\mathcal{R}$

In [134]:
f.write("<listOfReactions>\n")

18

### Metabolic Reactions $\mathcal{R_X}$ and $\mathcal{R_Y}$

In [135]:
stoich = 1.0
stoich_bool = False
for index, value in reactions_turnover['Covert ID'].items():
    if rdefba and value == 'LACUP':
        param_lower = 'T_LACUP_FBClower'
        param_upper = 'T_LACUP_FBCupper'
        fbc_parameters.append(param_lower)
        fbc_parameters.append(param_upper)
        f.write("<reaction id=\""+value+"\" reversible=\"false\" fbc:lowerFluxBound=\"" + param_lower + "\" fbc:upperFluxBound=\"" + param_upper + "\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(reactions_turnover['kcat_fwd/min'][index])+"\" ram:kcatBackward=\""+str(reactions_turnover['kcat_bwd/min'][index])+"\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    elif '<' in reactions_turnover['Reaction'][index]: # reversible
        f.write("<reaction id=\""+value+"\" reversible=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(reactions_turnover['kcat_fwd/min'][index])+"\" ram:kcatBackward=\""+str(reactions_turnover['kcat_bwd/min'][index])+"\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    else:
        f.write("<reaction id=\""+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(reactions_turnover['kcat_fwd/min'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    if pd.isna(reactions_turnover['fbc:geneProductAssociation'][index]): # spontaneous reactions
        pass
    else:
        f.write("  <fbc:geneProductAssociation fbc:id=\""+reactions_turnover['fbc:geneProductAssociation'][index]+"\">\n    <fbc:geneProductRef fbc:geneProduct=\""+reactions_turnover['fbc:geneProductAssociation'][index]+"\"/>\n  </fbc:geneProductAssociation>\n")
    f.write("  <listOfReactants>\n")
    for g in re.split(r'[+\s]\s*', reactions_turnover['Reaction'][index]):
        if '-' in g:
            f.write("  </listOfReactants>\n  <listOfProducts>\n")
        else:
            try:
                stoich = float(g)
                stoich_bool = True
            except ValueError:
                if g == "":
                    pass
                else:
                    if stoich_bool == False:
                        stoich = 1.0
                    f.write("    <speciesReference species=\""+g+"\" stoichiometry=\""+str(stoich)+"\" constant=\"true\"/>\n")
                    stoich_bool = False
    f.write("  </listOfProducts>\n</reaction>\n")

### Translation Reactions $\mathcal{R_P}$

In [136]:
for index, value in translation['ID'].items():
    if translation['ID'][index] == 'T_Quota_protein':
        prefix = ''
    elif pd.notnull(translation['Covert ID'][index]):
        prefix = 'E_'
    # Regulatory Proteins have no "Covert ID"
    else:
        if rdefba:
            prefix = 'RP_'
        else:
            break
    if rdefba:
        if pd.notnull(translation['Regulatory logic'][index]):
            param_upper = value + '_FBCupper'
            param_lower = value + '_FBClower'
            fbc_parameters.append(param_upper)
            fbc_parameters.append(param_lower)
            f.write("<reaction id=\""+value+"\" reversible=\"false\" fbc:lowerFluxBound=\"" + param_lower + "\" fbc:upperFluxBound=\"" + param_upper + "\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(translation['ram:kcatForward'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
        else:
            f.write("<reaction id=\""+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(translation['ram:kcatForward'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")    
    else:
        f.write("<reaction id=\""+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(translation['ram:kcatForward'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")    
    f.write("  <fbc:geneProductAssociation fbc:id=\"E_R\">\n    <fbc:geneProductRef fbc:geneProduct=\"E_R\"/>\n  </fbc:geneProductAssociation>\n")
    f.write("  <listOfReactants>\n    <speciesReference species=\"Z\" stoichiometry=\""+str(translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"ATP\" stoichiometry=\""+str(3*translation['Length'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n")
    f.write("  <listOfProducts>\n    <speciesReference species=\""+prefix+translation['ID'][index][2:]+"\" stoichiometry=\"1.0\" constant=\"true\"/>\n    <speciesReference species=\"AMP\" stoichiometry=\""+str(translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"PPI\" stoichiometry=\""+str(translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"ADP\" stoichiometry=\""+str(2*translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"PI\" stoichiometry=\""+str(2*translation['Length'][index])+"\" constant=\"true\"/>\n  </listOfProducts>\n</reaction>\n")   
    

### Quota_rest

In [137]:
f.write("<reaction id=\"R_Quota_rest\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"\" ram:kcatBackward=\"\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")    
stoich = 1.0
stoich_bool = False
f.write("  <listOfReactants>\n")
for g in re.split(r'[+\s]\s*', quota['Reaction'][quota.index[quota['ID'] == 'Quota_rest'].tolist()[0]]):
    if '-' in g:
        f.write("  </listOfReactants>\n  <listOfProducts>\n")
    else:
        try:
            stoich = float(g)
            stoich_bool = True
        except ValueError:
            if g == "":
                pass
            else:
                if g == "Biomass":
                    g = "Quota_rest"
                    stoich = 1.0
                elif stoich_bool == False:
                    stoich = 1.0
                f.write("    <speciesReference species=\""+g+"\" stoichiometry=\""+str(stoich)+"\" constant=\"true\"/>\n")
                stoich_bool = False
f.write("</listOfProducts>\n</reaction>\n")

30

### Degradation Reactions $\mathcal{R_{kd}}$

In [138]:
include_degradation = False

#### Enzymes

In [139]:
if include_degradation:
    for index, value in enzymes['id'].items():
        f.write("<reaction id=\"kd_"+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"NaN\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
        f.write("  <listOfReactants>\n    <speciesReference species=\""+value+"\" stoichiometry=\""+str(enzymes['degradationRate'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n</reaction>\n")

#### regulatory proteins

In [140]:
if rdefba and include_degradation:
    for index, value in rp['id'].items():
        f.write("<reaction id=\"kd_"+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"NaN\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
        f.write("  <listOfReactants>\n    <speciesReference species=\""+value+"\" stoichiometry=\""+str(rp['degradationRate'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n</reaction>\n")    

#### Quota

In [141]:
if include_degradation:
    for index, value in quota['ID'].items():
        f.write("<reaction id=\"kd_"+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"NaN\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
        f.write("  <listOfReactants>\n    <speciesReference species=\""+value+"\" stoichiometry=\""+str(quota['degradationRate'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n</reaction>\n")    

### Maintenance Reaction

In [142]:
f.write("<reaction id=\"Maintenance\" name=\"Maintenance\" reversible=\"false\">\n\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:reaction ram:kcatForward=\"\" ram:kcatBackward=\"\" ram:maintenanceScaling=\"0.14\"/>\n\t\t</ram:RAM>\n\t</annotation>\n\t<listOfReactants>\n\t\t<speciesReference species=\"ATP\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfReactants>\n\t<listOfProducts>\n\t\t<speciesReference species=\"ADP\" stoichiometry=\"1\" constant=\"true\"/>\n\t\t<speciesReference species=\"PI\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfProducts>\n</reaction>\n")

553

In [143]:
f.write("</listOfReactions>\n\n")

20

## List of Gene Products

### Enzymes (including Ribosome)

In [144]:
f.write("<fbc:listOfGeneProducts>\n")

25

In [145]:
for index, value in enzymes['id'].items():
    if enzymes['Gene'][index]:
        f.write("  <fbc:geneProduct fbc:id=\""+value+"\" fbc:label=\""+enzymes['Gene'][index]+"_"+value[2:]+"\" fbc:associatedSpecies=\""+value+"\"/>\n")
# Gene_Protein in order to have unique fbc:label values

### Regulatory Proteins

In [146]:
if rdefba:
    for index, value in rp['id'].items():
        f.write("  <fbc:geneProduct fbc:id=\""+value+"\" fbc:label=\""+rp['Gene'][index]+"\" fbc:associatedSpecies=\""+value+"\"/>\n")

In [147]:
f.write("</fbc:listOfGeneProducts>\n\n")

27

# List of Qualitative Species

In [148]:
if rdefba:
    f.write("<qual:listOfQualitativeSpecies>\n")
    for index, value in qualspec['id'].items():
        f.write("  <qual:qualitativeSpecies qual:id=\""+value+"\" qual:maxLevel=\""+str(qualspec['max'][index])+"\" qual:compartment=\"cytosol\" qual:constant=\"false\"/>\n")
    f.write("</qual:listOfQualitativeSpecies>\n\n")

# List of Rules

In [149]:
if rdefba:
    f.write("<listOfRules>\n")

## FBC rules

In [150]:
# translation always irreversible
if rdefba:
    for index, value in translation['ID'].items():
        if pd.notnull(translation['Regulatory logic'][index]):
            f.write("  <assignmentRule variable=\"" + value + "_FBCupper\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>NaN</ci>\n        <ci>" + translation['control variable'][index] + "</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")
            f.write("  <assignmentRule variable=\"" + value + "_FBClower\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>epsilon_trans</ci>\n        <ci>" + translation['control variable'][index] + "</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")         
        

In [151]:
# Lactate (LACUP) -- spontaneous, but regulated
if rdefba:
    qual_parameters.append('GL_neg')
    f.write("  <assignmentRule variable=\"T_LACUP_FBClower\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>epsilon_trans</ci>\n        <ci>GL_neg</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")         
    f.write("  <assignmentRule variable=\"T_LACUP_FBCupper\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>NaN</ci>\n        <ci>GL_neg</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")


## Qual Rules

In [152]:
if rdefba:
    for qual in qualspec['id']:
        f.write("<assignmentRule variable=\"" + qual + "\">\n  <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n    <ci>qual_con_" + qual + "</ci>\n  </math>\n</assignmentRule>\n")
        

## Boolean Rules

In [153]:
if rdefba:
    for index, value in rules['variable'].items():
        qual_parameters.append(value)
        f.write("<assignmentRule variable=\"" + value + "\">\n  <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n    <apply>\n      " + rules['relation'][index] + "\n")
        for i in rules['indicators'][index]:
            qual_parameters.append(i)
            f.write("      <ci>" + i + "</ci>\n")
        f.write("    </apply>\n  </math>\n</assignmentRule>\n")

In [154]:
if rdefba:
    f.write("</listOfRules>\n\n")

# Events

In [155]:
if rdefba:
    f.write("<listOfEvents>\n")
    for index, value in events['id'].items():
        f.write("<event id=\"" + value + "\" useValuesFromTriggerTime=\"true\">\n  <trigger persistent=\"true\" initialValue=\"true\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n")
        f.write("      <apply>\n        <" + events['relation'][index] + "/>\n")
        if len(events['variable'][index]) > 1: # multiple variables
            f.write("        <apply>\n          <plus/>\n")
            for var in events['variable'][index]:
                f.write("          <ci>" + var + "</ci>\n")
            f.write("        </apply>\n")
        else:
            f.write("        <ci>" + events['variable'][index][0] + "</ci>\n")
        f.write("        <ci>"+events['threshold'][index]+"</ci>\n      </apply>\n    </math>\n  </trigger>\n  <listOfEventAssignments>\n")
        for i, assignment in enumerate(events['assignment'][index]):
            qual_parameters.append(assignment)
            f.write("    <eventAssignment variable=\""+assignment+"\">\n      <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n        <cn type=\"integer\">"+events['value'][index][i]+"</cn>\n      </math>\n    </eventAssignment>\n")
        f.write("  </listOfEventAssignments>\n</event>\n")
    f.write("</listOfEvents>\n\n")

# List of Parameters

In [156]:
# filter duplicates
qual_parameters = list(set(qual_parameters))

In [157]:
if rdefba:
    f.write("<listOfParameters>\n")
    for index, value in parameters.items():
        f.write("  <parameter id=\"" + index + "\" constant=\"true\" value=\"" + str(value) + "\"/>\n")
    for fbc_param in fbc_parameters:
        f.write("  <parameter id=\"" + fbc_param + "\" value=\"NaN\" constant=\"false\" />\n")
    for qual_param in qual_parameters:
        f.write("  <parameter id=\"" + qual_param + "\" value=\"NaN\" constant=\"false\" />\n")
    f.write("</listOfParameters>\n")

## Last lines

In [158]:
f.write("</model>\n")
f.write("</sbml>")

7

In [159]:
f.close()